# Problems
- What makes a movie good, bad? How would you define those? (Classification)
- Does the popularity of the casts, crews and the movie itself affects the quality of the movie?
- Logistic Regression vs Decision Tree vs Random Forest Classifier vs XGB Classifier vs SVM Classifier

# Import necessary libraries

In [3]:
# For data processing
import numpy as np
import pandas as pd

# For API usage
import requests as r

# For progress bar
from tqdm import tqdm

# Helper functions for data extraction

In [4]:
def compute_top_avg(df, col, p):
    """
    NOTE: This function is deprecated. In the main function, 
          we use the compute_top_five_avg function instead.
    
    Computes the average value of the top p% of a 
    certain column (col) in the DataFrame (df).
    
    Minimum entries to be included is 1.
    """
    
    try:
        # Sort the column in descending order
        top = df[col].sort_values(ascending=False)
        
        # Selects the top p%
        top = top[:max(int(len(df) * p), 1)]
        
        return top.sum() / len(top)
    except:
        return np.nan
    
    
def compute_top_five_avg(df, col):
    """
    Computes the average of the top 5 values of a 
    column (col) in the DataFrame (df).

    If there are less than 5 entries in df, it will use
    the top len(df) entries in df.
    """
    
    try:
        # Sort the column in descending order
        top = df[col].sort_values(ascending=False)
        
        # Selects the top min(len(df), 5) entries
        top = top[:min(len(df), 5)]
        return top.sum() / len(top)
    except:
        return np.nan
    

def get_popularity_stats(resp, col):
    """
    Gets the column's (col):
        1. Average popularity value of the top p% entries
        2. Total popularity value of all the entries
        3. Highest popularity value
    from the response (resp).
    
    Returns a dictionary containing the 3 data above.
    """
    
    result = {}
    
    try:
        data = pd.DataFrame(resp.json()[col])
        
        # Computes the average popularity value of the top p% entries
        try:
            result['avg'] = compute_top_five_avg(data, 'popularity')
        except:
            result['avg'] = np.nan
        
        # Computes the total popularity value of all the entries
        try:
            result['sum'] = data['popularity'].sum()
        except:
            result['sum'] = np.nan 
        
        # Computes the highest popularity value
        try:
            result['top'] = data['popularity'].max()
        except:
            result['top'] = np.nan

    except KeyError:
        result['avg'] = np.nan
        result['sum'] = np.nan
        result['top'] = np.nan
        
    return result

def getdecade(year):
    if year >= 2020:
        return '2020s'
    elif year >= 2010:
        return '2010s'
    elif year >=2000:
        return '2000s'
    elif year >=1990:
        return '1990s'
    elif year >=1980:
        return '1980s'
    elif year >=1970:
        return '1970s'
    elif year >=1960:
        return '1960s'
    else:
        return 'Movies before 1960s'

def getgenres(genredict):
    genres = []
    for i in genredict:
        genres.append(i['name'])
    return genres

In [43]:
API_KEY = 'db6bd34e31c99738cfb114ed7ad6d566'


def get_movies_data_init():
    """
    Returns a DF containing:
        1. Title
        2. ID
        3. Popularity value
        4. Vote count
        5. Vote average (or rating)
        6. Release date
        7. Original Language of movie
    for each movie.
    """
    
    # Create empty DF to store the movie data
    movie = pd.DataFrame()
    
    # Initial response to get the number of pages
    resp = r.get("https://api.themoviedb.org/3/discover/movie?" \
                 f"api_key={API_KEY}&language=en-US&sort_by=revenue.desc&" \
                 "include_adult=false&include_video=false&page=1")
    total_pages = resp.json()['total_pages']
    
    # Relevant movie columns
    cols = ['title', 'id', 'popularity', 'vote_count', 'vote_average','release_date', 'original_language']
    
    for i in tqdm(range(1,total_pages + 1)):
        resp = r.get(f"https://api.themoviedb.org/3/discover/movie?api_key={API_KEY}&language=en-US&sort_by=revenue.desc&include_adult=false&include_video=false&page={i}")
        
        # Select the relevant columns
        data = pd.DataFrame(resp.json()['results'])[cols]
        movie = movie.append(data)
    
    movie = movie.reset_index(drop=True)
    
    return movie


def get_movies_stats(movie_df):
    """
    Gets the casts' and crews':
        1. Average popularity value of the top p% entries
        2. Total popularity value of all the entries
        3. Highest popularity value
    for each movie 
    and also:
        - Budget
        - Revenue
        - Genres
        - Year released
        - Decade released of the movie
    
    Returns a DF containing the above data.
    """
    
    cols = ['top_casts_popularity_avg', 'casts_popularity_sum', 'top_cast_popularity',
            'top_crews_popularity_avg', 'crews_popularity_sum', 'top_crew_popularity', 
            'budget','revenue','genres', 'year_released', 'decade_released']
    
    rows = []
    
    for i in tqdm(range(len(movie_df))):
        row = []
        movie_id = movie_df['id'][i]
        
        resp = r.get(f'https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={API_KEY}&language=en-US')
        
        # Get casts' popularity statistics from the movie
        casts_stats = get_popularity_stats(resp, 'cast')
        
        # Append to row
        row.append(casts_stats['avg'])
        row.append(casts_stats['sum'])
        row.append(casts_stats['top'])
        
        # Get crews' popularity statistics from the movie
        crews_stats = get_popularity_stats(resp, 'crew')
        
        # Append to row
        row.append(crews_stats['avg'])
        row.append(crews_stats['sum'])
        row.append(crews_stats['top'])
        
        # Get the movie's other data that is not found in all movies
        resp = r.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={API_KEY}&language=en-US')
        
        try:
            budget = resp.json()['budget']
            if budget == 0:
                row.append(np.nan)
            else:
                row.append(budget)
        except KeyError:
            row.append(np.nan)
        
        try:
            revenue = resp.json()['revenue']
            if revenue == 0:
                row.append(np.nan)
            else:
                row.append(revenue)
        except KeyError:
            row.append(np.nan)
            
        try:
            genres = getgenres(resp.json()['genres'])
            row.append(genres)
        except:
            row.append(np.nan)
        
        try:
            yearmovie = int(resp.json()['release_date'][0:4])
            decadenum = getdecade(yearmovie)
            row.append(yearmovie)
            row.append(decadenum)
        except:
            row.append(np.nan)
            row.append(np.nan)
        
        rows.append(row)
    
    # Create a DF containing all the data for all movies
    stats_df = pd.DataFrame(rows, columns=cols)

    return stats_df


def get_movies_data_all():
    """
    Returns the complete DF with the necessary columns using helper functions above.
    """
    
    # Get the initial DF
    movie_df = get_movies_data_init()
    
    # Get the movies' statistics
    stats_df = get_movies_stats(movie_df)
    
     # Concatenate both DFs
    complete_df = pd.concat([movie_df, stats_df], axis=1)
    
     # Rearrange the columns so that vote_count and vote_average lies in the last 2 columns
    cols = complete_df.columns.tolist()
    cols = cols[:3] + cols[5:] + cols[3:5]
    
    complete_df = complete_df[cols]
    
    return complete_df

In [44]:
df = get_movies_data_all()
df.to_csv('full-movie-dataset.csv')

100%|██████████| 10000/10000 [18:10<00:00,  9.17it/s]


In [45]:
df

,title,id,popularity,release_date,original_language,top_casts_popularity_avg,casts_popularity_sum,top_cast_popularity,top_crews_popularity_avg,crews_popularity_sum,top_crew_popularity,budget,revenue,genres,year_released,decade_released,vote_count,vote_average
0,Avengers: Endgame,299534,283.822,2019-04-24,en,30.39520,755.450,47.613,9.4208,473.269,9.929,356000000.0,2.797801e+09,"[Adventure, Science Fiction, Action]",2019.0,2010s,17799,8.3
1,Avatar,19995,102.112,2009-12-10,en,10.26480,108.804,13.136,4.0792,878.826,4.413,237000000.0,2.787965e+09,"[Action, Adventure, Fantasy, Science Fiction]",2009.0,2000s,23178,7.5
2,Titanic,597,86.808,1997-11-18,en,11.95260,201.581,25.466,4.2744,122.649,4.413,200000000.0,2.187464e+09,"[Drama, Romance]",1997.0,1990s,18982,7.9
3,Star Wars: The Force Awakens,140607,53.930,2015-12-15,en,16.01320,391.303,23.880,5.7828,235.947,11.482,245000000.0,2.068224e+09,"[Action, Adventure, Science Fiction, Fantasy]",2015.0,2010s,15675,7.4
4,Avengers: Infinity War,299536,299.524,2018-04-25,en,30.39520,528.712,47.613,7.7592,501.005,9.929,300000000.0,2.046240e+09,"[Adventure, Action, Science Fiction]",2018.0,2010s,21489,8.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,"Madrid, Costa Fleming",446100,1.340,1976-04-26,es,1.85740,38.620,2.548,0.6948,7.074,0.980,NaN,3.325280e+05,[Comedy],1976.0,1970s,0,0.0
9996,El crack cero,613092,5.124,2019-10-04,es,1.57940,23.391,1.960,1.6264,11.732,1.788,NaN,3.323560e+05,"[Drama, Thriller]",2019.0,2010s,20,7.0
9997,Private Resort,17824,10.694,1985-05-03,en,9.65500,61.643,31.784,1.3696,14.068,1.620,NaN,3.318160e+05,[Comedy],1985.0,1980s,145,5.7
9998,Salome's Last Dance,70842,6.976,1988-05-06,en,1.67580,32.035,2.250,1.6156,42.472,1.960,NaN,3.314690e+05,[Drama],1988.0,1980s,19,6.5


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   title                     10000 non-null  object 
 1   id                        10000 non-null  int64  
 2   popularity                10000 non-null  float64
 3   release_date              9991 non-null   object 
 4   original_language         10000 non-null  object 
 5   top_casts_popularity_avg  9937 non-null   float64
 6   casts_popularity_sum      9937 non-null   float64
 7   top_cast_popularity       9937 non-null   float64
 8   top_crews_popularity_avg  9916 non-null   float64
 9   crews_popularity_sum      9916 non-null   float64
 10  top_crew_popularity       9916 non-null   float64
 11  budget                    7128 non-null   float64
 12  revenue                   9991 non-null   float64
 13  genres                    9991 non-null   object 
 14  year_re

In [47]:
# Data Cleaning
cleaned_df = df.drop(columns=['title', 'id', 'release_date']).dropna()
cleaned_df.to_csv('cleaned-movie-dataset.csv')